<a href="https://colab.research.google.com/github/ColaXww/neo4j-nlp/blob/master/hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
ls

adc.json  drive/  sample_data/


In [0]:
! uname -a  # 查看系统  x86_64 GNU/Linux
! python --version  # 查看python版本 Python 3.6.7
! nvcc --version # 查看cuda版本 release 9.2, V9.2.148

Linux 41feaa6c47ac 4.14.137+ #1 SMP Thu Aug 8 02:47:02 PDT 2019 x86_64 x86_64 x86_64 GNU/Linux
Python 3.6.8
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
! pip3 install torch torchvision  # 省事

In [0]:
! pip install torch torchvision  # 省事

In [0]:
import torch

In [0]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED) #为CPU设置随机种子
torch.cuda.manual_seed(SEED)#为GPU设置随机种子
torch.backends.cudnn.deterministic = True  #在程序刚开始加这条语句可以提升一点训练速度，没什么额外开销。

TEXT = data.Field(tokenize='spacy')#torchtext.data.Field : 用来定义字段的处理方法（文本字段，标签字段）
LABEL = data.LabelField(dtype=torch.float)
#首先，我们要创建两个Field 对象：这两个对象包含了我们打算如何预处理文本数据的信息。
#spaCy:英语分词器,类似于NLTK库，如果没有传递tokenize参数，则默认只是在空格上拆分字符串。
#LabelField是Field类的一个特殊子集，专门用于处理标签。 

from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
print(vars(train_data.examples[0])) #可以查看数据集长啥样子

{'text': ['Just', 'saying', 'you', "'ve", 'got', 'a', 'movie', 'about', 'John', 'Holmes', 'is', 'a', 'guarantee', 'to', 'get', 'some', 'folks', 'in', 'front', 'of', 'the', 'screen', ',', 'but', 'writer', '/', 'director', 'James', 'Cox', 'delivers', 'oh', 'so', 'much', 'more', '.', 'A', '"', 'Rashamon', '"', 'of', 'the', 'sleazy', 'Hollywood', 'set', ',', 'the', 'film', 'splitters', 'the', 'July', '1981', 'Wonderland', 'murders', 'through', 'a', 'variety', 'of', 'angles', '(', 'and', 'film', 'stocks', ')', ',', 'but', 'mostly', 'through', 'the', 'filter', 'of', 'John', 'Holmes', "'", 'coked', 'out', 'weasel', 'brain', '.', 'In', 'a', 'film', 'full', 'of', 'bad', 'guys', 'Holmes', 'is', 'either', 'the', 'most', 'vile', ',', 'the', 'most', 'pathetic', 'or', 'both', '.', 'Several', 'versions', 'of', 'the', 'story', 'emerge', 'and', 'merge', 'as', 'Cox', 'flashes', 'jump', 'cuts', 'and', 'twisting', 'title', 'cards', 'amid', 'effects', 'and', 'emoting', '.', 'The', 'dialogue', 'is', 'fast',

In [0]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(SEED)) #默认split_ratio=0.7

In [0]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [0]:
# TEXT.build_vocab(train_data, max_size=25000)
# LABEL.build_vocab(train_data)
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
#从预训练的词向量（vectors） 中，将当前(corpus语料库)词汇表的词向量抽取出来，构成当前 corpus 的 Vocab（词汇表）。
#预训练的 vectors 来自glove模型，每个单词有100维。glove模型训练的词向量参数来自很大的语料库，
#而我们的电影评论的语料库小一点，所以词向量需要更新，glove的词向量适合用做初始化参数。
LABEL.build_vocab(train_data) 

.vector_cache/glove.6B.zip: 862MB [06:29, 2.22MB/s]                           
100%|█████████▉| 399801/400000 [00:21<00:00, 18222.34it/s]

In [17]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [18]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203162), (',', 193173), ('.', 165972), ('and', 109558), ('a', 109170), ('of', 100931), ('to', 94128), ('is', 76203), ('in', 61384), ('I', 54697), ('it', 53930), ('that', 49242), ('"', 44552), ("'s", 43479), ('this', 42568), ('-', 37131), ('/><br', 36029), ('was', 35281), ('as', 30448), ('with', 29971)]


In [19]:
print(TEXT.vocab.stoi) #语料库单词频率越高，索引越靠前。前两个默认为unk和pad。

defaultdict(<function _default_unk_index at 0x7fc50918b6a8>, {'<unk>': 0, '<pad>': 1, 'the': 2, ',': 3, '.': 4, 'and': 5, 'a': 6, 'of': 7, 'to': 8, 'is': 9, 'in': 10, 'I': 11, 'it': 12, 'that': 13, '"': 14, "'s": 15, 'this': 16, '-': 17, '/><br': 18, 'was': 19, 'as': 20, 'with': 21, 'movie': 22, 'for': 23, 'film': 24, 'The': 25, 'but': 26, '(': 27, "n't": 28, ')': 29, 'on': 30, 'you': 31, 'are': 32, 'not': 33, 'have': 34, 'his': 35, 'be': 36, 'he': 37, 'one': 38, 'at': 39, 'by': 40, '!': 41, 'all': 42, 'an': 43, 'who': 44, 'they': 45, 'from': 46, 'like': 47, 'so': 48, 'her': 49, 'about': 50, 'or': 51, "'": 52, 'has': 53, 'out': 54, 'just': 55, 'It': 56, 'do': 57, '?': 58, 'some': 59, 'good': 60, 'more': 61, 'very': 62, 'would': 63, 'up': 64, 'what': 65, 'This': 66, 'there': 67, 'time': 68, 'can': 69, 'which': 70, 'when': 71, 'she': 72, 'really': 73, 'only': 74, 'story': 75, 'had': 76, 'were': 77, 'if': 78, 'their': 79, 'see': 80, 'even': 81, 'no': 82, 'my': 83, 'does': 84, 'me': 85, 'd

In [20]:
print(TEXT.vocab.itos[:10]) #查看TEXT单词表

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#相当于把样本划分batch，只是多做了一步，把相等长度的单词尽可能的划分到一个batch，不够长的就用padding。
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

In [22]:
next(iter(train_iterator)).label 
next(iter(train_iterator)).text 
next(iter(train_iterator))#多运行一次可以发现一条评论的单词长度会变


[torchtext.data.batch.Batch of size 64]
	[.text]:[torch.cuda.LongTensor of size 866x64 (GPU 0)]
	[.label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]

In [23]:
next(iter(train_iterator)).text 

tensor([[  14,   11,   11,  ...,  149, 2509,   66],
        [1229,  266, 1075,  ...,    0,   90,   24],
        [  15,    0,   21,  ...,    0,   56,    9],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class WordAVGModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        #初始化参数，
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        #vocab_size=词汇表长度，embedding_dim=每个单词的维度
        #padding_idx：如果提供的话，输出遇到此下标时用零填充。这里如果遇到padding的单词就用0填充。
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        #output_dim输出的维度，一个数就可以了，=1
        
    def forward(self, text):
        embedded = self.embedding(text) 
        #text下面会指定，为一个batch的数据
        # embedded = [sent len, batch size, emb dim] 
        # sent len：一条评论的单词数 
        # batch size：一个batch有多少条评论
        # emb dim：一个单词的维度
        # 假设[sent len, batch size, emb dim]=（1000，64，100）
        #这个代码我猜测进行了运算：（text：1000，64，25000）*（self.embedding：1000，25000，100）= （1000，64，100）
        
        embedded = embedded.permute(1, 0, 2) 
        # [batch size, sent len, emb dim]更换顺序
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        # [batch size, embedding_dim] 把单词长度的维度压扁为1，并降维
        
        return self.fc(pooled)  
        #（batch size, embedding_dim）*（embedding_dim, output_dim）=（batch size,output_dim）

In [0]:
INPUT_DIM = len(TEXT.vocab) #25002
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] 
#PAD_IDX = 1 为pad的索引

model = WordAVGModel(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [26]:
def count_parameters(model): #统计参数，可以不用管
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [27]:
pretrained_embeddings = TEXT.vocab.vectors 
model.embedding.weight.data.copy_(pretrained_embeddings) #遇到_的语句直接替换，不需要另外赋值=
#把上面vectors="glove.6B.100d"取出的词向量作为初始化参数，数量为25000*100个参数


tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.4654,  0.0499,  0.4693,  ..., -0.1296,  1.3465,  0.0561],
        [-0.5045,  1.0418,  0.3279,  ...,  0.6332, -0.4239,  0.6381],
        [ 0.5361, -0.2952, -0.4806,  ...,  0.0840, -0.5090,  0.5193]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token] #UNK_IDX=0

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
#词汇表25002个单词，前两个unk和pad也需要初始化

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters()) #定义优化器
criterion = nn.BCEWithLogitsLoss()  #定义损失函数，这个BCEWithLogitsLoss特殊情况，二分类损失函数
model = model.to(device) #送到gpu上去
criterion = criterion.to(device) #送到gpu上去

In [0]:
def binary_accuracy(preds, y): #计算准确率
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    #.round函数：四舍五入
    
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    
    epoch_loss = 0
    epoch_acc = 0
    total_len = 0
    model.train() #model.train()代表了训练模式
    #这步一定要加，是为了区分model训练和测试的模式的。
    #有时候训练时会用到dropout、归一化等方法，但是测试的时候不能用dropout等方法。
    
    
    
    for batch in iterator: #iterator为train_iterator
        optimizer.zero_grad() #加这步防止梯度叠加
        
        predictions = model(batch.text).squeeze(1)
        #batch.text 就是上面forward函数的参数text
        #压缩维度，不然跟batch.label维度对不上
        
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        
        loss.backward() #反向传播
        optimizer.step() #梯度下降
        
        epoch_loss += loss.item() * len(batch.label)
        #loss.item()已经本身除以了len(batch.label)
        #所以得再乘一次，得到一个batch的损失，累加得到所有样本损失。
        
        epoch_acc += acc.item() * len(batch.label)
        #（acc.item()：一个batch的正确率） *batch数 = 正确数
        #train_iterator所有batch的正确数累加。
        
        total_len += len(batch.label)
        #计算train_iterator所有样本的数量，不出意外应该是17500
        
    return epoch_loss / total_len, epoch_acc / total_len
    #epoch_loss / total_len ：train_iterator所有batch的损失
    #epoch_acc / total_len ：train_iterator所有batch的正确率

In [0]:
def evaluate(model, iterator, criterion):
     
    
    epoch_loss = 0
    epoch_acc = 0
    total_len = 0
    
    model.eval()
    #转换成测试模式，冻结dropout层或其他层。
    
    with torch.no_grad():
        for batch in iterator: 
            #iterator为valid_iterator
            
            #没有反向传播和梯度下降
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            
            epoch_loss += loss.item() * len(batch.label)
            epoch_acc += acc.item() * len(batch.label)
            total_len += len(batch.label)
    model.train() #调回训练模式   
    
    return epoch_loss / total_len, epoch_acc / total_len

In [0]:
import time 

def epoch_time(start_time, end_time):  #查看每个epoch的时间
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
N_EPOCHS = 10

best_valid_loss = float('inf') #无穷大

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss: #只要模型效果变好，就存模型
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'wordavg-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 6s
	Train Loss: 0.686 | Train Acc: 59.07%
	 Val. Loss: 0.625 |  Val. Acc: 71.84%
Epoch: 02 | Epoch Time: 0m 6s
	Train Loss: 0.646 | Train Acc: 72.20%
	 Val. Loss: 0.513 |  Val. Acc: 75.49%
Epoch: 03 | Epoch Time: 0m 6s
	Train Loss: 0.574 | Train Acc: 78.23%
	 Val. Loss: 0.459 |  Val. Acc: 79.80%
Epoch: 04 | Epoch Time: 0m 6s
	Train Loss: 0.503 | Train Acc: 81.97%
	 Val. Loss: 0.428 |  Val. Acc: 82.89%
Epoch: 05 | Epoch Time: 0m 6s
	Train Loss: 0.442 | Train Acc: 85.10%
	 Val. Loss: 0.414 |  Val. Acc: 84.65%
Epoch: 06 | Epoch Time: 0m 6s
	Train Loss: 0.391 | Train Acc: 87.62%
	 Val. Loss: 0.419 |  Val. Acc: 85.95%
Epoch: 07 | Epoch Time: 0m 6s
	Train Loss: 0.354 | Train Acc: 88.64%
	 Val. Loss: 0.431 |  Val. Acc: 86.49%
Epoch: 08 | Epoch Time: 0m 6s
	Train Loss: 0.322 | Train Acc: 89.74%
	 Val. Loss: 0.447 |  Val. Acc: 86.99%
Epoch: 09 | Epoch Time: 0m 6s
	Train Loss: 0.295 | Train Acc: 90.60%
	 Val. Loss: 0.464 |  Val. Acc: 87.49%
Epoch: 10 | Epoch Time: 0m 6

In [35]:
model.load_state_dict(torch.load("wordavg-model.pt"))
#用保存的模型参数预测数据

<All keys matched successfully>

In [0]:
import spacy  #分词工具，跟NLTK类似
nlp = spacy.load('en')

def predict_sentiment(sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]#分词
    indexed = [TEXT.vocab.stoi[t] for t in tokenized] 
    #sentence的索引
    
    tensor = torch.LongTensor(indexed).to(device) #seq_len
    tensor = tensor.unsqueeze(1) 
    #seq_len * batch_size（1）
    
    prediction = torch.sigmoid(model(tensor))
    #tensor与text一样的tensor
    
    return prediction.item()



In [37]:
predict_sentiment("I love This film bad ")

0.04145227000117302

In [38]:
predict_sentiment("This film is great")

1.0

In [0]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, 
                           bidirectional=bidirectional, dropout=dropout)
        #embedding_dim：每个单词维度
        #hidden_dim：隐藏层维度
        #num_layers：神经网络深度，纵向深度
        #bidirectional：是否双向循环RNN
        #这个自己先得理解LSTM各个维度，不然容易晕
        
        
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.dropout(self.embedding(text)) #[sent len, batch size, emb dim]
        output, (hidden, cell) = self.rnn(embedded)
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)) # [batch size, hid dim * num directions]
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
            N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [41]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [42]:
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.4654,  0.0499,  0.4693,  ..., -0.1296,  1.3465,  0.0561],
        [-0.5045,  1.0418,  0.3279,  ...,  0.6332, -0.4239,  0.6381],
        [ 0.5361, -0.2952, -0.4806,  ...,  0.0840, -0.5090,  0.5193]])


In [0]:
optimizer = optim.Adam(model.parameters())
model = model.to(device)

In [44]:
N_EPOCHS = 5
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 5m 8s
	Train Loss: 0.679 | Train Acc: 56.35%
	 Val. Loss: 0.632 |  Val. Acc: 64.67%
Epoch: 02 | Epoch Time: 5m 9s
	Train Loss: 0.589 | Train Acc: 69.54%
	 Val. Loss: 0.466 |  Val. Acc: 81.16%
Epoch: 03 | Epoch Time: 5m 13s
	Train Loss: 0.693 | Train Acc: 51.07%
	 Val. Loss: 0.690 |  Val. Acc: 54.55%
Epoch: 04 | Epoch Time: 5m 14s
	Train Loss: 0.662 | Train Acc: 59.30%
	 Val. Loss: 0.593 |  Val. Acc: 70.23%
Epoch: 05 | Epoch Time: 5m 14s
	Train Loss: 0.418 | Train Acc: 82.41%
	 Val. Loss: 0.337 |  Val. Acc: 86.91%


In [45]:
model.load_state_dict(torch.load('lstm-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.363 | Test Acc: 85.53%


In [0]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, 
                 filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        text = text.permute(1, 0) # [batch size, sent len]
        embedded = self.embedding(text) # [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1) # [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]


model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
model = model.to(device)

In [48]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'CNN-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 32s
	Train Loss: 0.654 | Train Acc: 61.17%
	 Val. Loss: 0.530 |  Val. Acc: 75.93%
Epoch: 02 | Epoch Time: 0m 32s
	Train Loss: 0.432 | Train Acc: 80.19%
	 Val. Loss: 0.357 |  Val. Acc: 85.20%
Epoch: 03 | Epoch Time: 0m 31s
	Train Loss: 0.308 | Train Acc: 87.14%
	 Val. Loss: 0.326 |  Val. Acc: 86.67%
Epoch: 04 | Epoch Time: 0m 31s
	Train Loss: 0.223 | Train Acc: 91.14%
	 Val. Loss: 0.312 |  Val. Acc: 87.33%
Epoch: 05 | Epoch Time: 0m 32s
	Train Loss: 0.159 | Train Acc: 94.04%
	 Val. Loss: 0.343 |  Val. Acc: 86.60%


In [49]:
model.load_state_dict(torch.load('CNN-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.332 | Test Acc: 85.84%
